#### Implementation of Spatio-Temporal Graph Convolutional Networks

In [ ]:
import pandas as pd
import pickle as pk
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
num_timesteps_input = 12
num_timesteps_output = 12

epochs = 100
batch_size = 100

use_gpu = False

if use_gpu and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
def load_adjacent_pickle(filename):
    """
    :param filename: the filename of the pickle file
    :return: ID_mapping: the mapping from node number in CSV to [0, num_nodes]
             A: the normalized adjacency matrix
    """
    with open(filename, "rb") as f:
        data = pk.load(f)
    ID_mapping = data[0]
    A = data[1]
    return ID_mapping, A


def load_csv(filename, ID_mapping):
    """
    :param filename: the filename of the CSV file
    :param ID_mapping: the mapping from node number in CSV to [0, num_nodes]
    :return: data: the unnormalized np array in shape (num_nodes, num_timesteps, num_features=1)
             num_nodes: number of nodes in graph
             num_timesteps: number of timesteps
    """
    df = pd.read_csv(filename)
    num_nodes = len(df.columns) - 1
    num_timesteps = len(df)
    data = [None] * num_nodes

    for (col_name, col_data) in df.iteritems():
        if col_name == "timestamp":
            continue
        node_id = ID_mapping[col_name]
        data[node_id] = col_data.values.tolist()
        for i in range(num_timesteps):
            data[node_id][i] = [data[node_id][i]]

    data = np.array(data)
    return data, num_nodes, num_timesteps


def modify_abnormal_data(unnormalized_data, num_nodes, num_timesteps):
    """
    :param unnormalized_data: the unnormalized np array in shape (num_nodes, num_timesteps, num_features=1)
    :param num_nodes: number of nodes in graph
    :param num_timesteps: number of timesteps
    :return: raw_data: the normalized data
    """

    for i in range(num_nodes):
        for j in range(num_timesteps):
            assert len(unnormalized_data[i][j]) == 1
            if abs(unnormalized_data[i][j][0]) <= 1e-6:
                unnormalized_data[i][j][0] = (unnormalized_data[i][j - 1][0] + unnormalized_data[i][j + 1][0]) / 2

    mean = np.mean(unnormalized_data)
    std = np.std(unnormalized_data)
    return unnormalized_data, mean, std


def generate_training_data(raw_data, num_timesteps, mean, std):
    X_train, Y_train = [], []

    for time in range(num_timesteps - num_timesteps_input - num_timesteps_output + 1):
        X_train.append(raw_data[:, time:time + num_timesteps_input])
        Y_train.append(raw_data[:, time + num_timesteps_input:time + num_timesteps_input + num_timesteps_output])

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    Y_train = Y_train.reshape((Y_train.shape[0], Y_train.shape[1], Y_train.shape[2]))

    X_train = (X_train - mean) / std
    Y_train = (Y_train - mean) / std
    return X_train, Y_train


def generate_test_data(raw_data, mean, std):
    X_test = []

    for time in range(0, raw_data.shape[1], num_timesteps_output):
        X_test.append(raw_data[:, time:time + num_timesteps_output])

    X_test = np.array(X_test)
    X_test = (X_test - mean) / std
    return X_test


In [ ]:
ID_mapping, A = load_adjacent_pickle("adjacent.pkl")
unnormalized_data, num_nodes, num_timesteps = load_csv("train.csv", ID_mapping)
unnormalized_data, mean, std = modify_abnormal_data(unnormalized_data, num_nodes, num_timesteps)
X_train, Y_train = generate_training_data(unnormalized_data, num_timesteps, mean, std)
num_samples = X_train.shape[0]

A = torch.from_numpy(A).double().to(device=device)
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train)

# shape of A: (num_nodes, num_nodes)
# shape of X_train and Y_train: (num_samples, num_nodes, num_timesteps_input/output, 1)

validation_split_line = int(X_train.shape[0] * 0.8)


In [ ]:
unnormalized_data, _, _ = load_csv("test.csv", ID_mapping)
X_test = generate_test_data(unnormalized_data, mean, std)
X_test = torch.from_numpy(X_test)


In [ ]:
class Temporal_Gated_Conv(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size):
        super(Temporal_Gated_Conv, self).__init__()
        self.output_channels = output_channels
        self.conv = nn.Conv2d(input_channels, 2 * output_channels, (1, kernel_size))
        self.batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X):
        """
        :param X: input data with shape (batch_size, num_nodes, num_timesteps, input_channels)
        :return: output data with shape (batch_size, num_nodes, num_timesteps(new), output_channels)
        """
        # Step 1: convert X into shape (batch_size, num_channels, num_nodes, num_timesteps)
        X = X.permute(0, 3, 1, 2)
        # Step 2: perform convolution, split it into P and Q
        conv_result = self.conv(X)
        P = conv_result[:, :self.output_channels, :, :]
        Q = conv_result[:, self.output_channels:, :, :]
        # Step 3: calculate result, convert it into shape (batch_size, num_nodes, num_timesteps, num_channels)
        result = P * torch.sigmoid(Q)
        result = result.permute(0, 2, 3, 1)
        result = self.batch_norm(result)
        return result


class ST_Conv_Block(nn.Module):
    def __init__(self, input_channels, temporal_channels, spatio_channels, output_channels, temporal_kernel_size):
        """
        :param input_channels: number of input channels
        :param temporal_channels: number of output channels of the first temporal-gated-conv block
        :param spatio_channels: number of output channels of the spatio graph-conv block
        :param output_channels: number of output channels of the second temporal-gated-conv block
        """
        super(ST_Conv_Block, self).__init__()
        self.temporal_gated_conv1 = Temporal_Gated_Conv(input_channels, temporal_channels, temporal_kernel_size)
        self.theta = nn.Parameter(torch.zeros((temporal_channels, spatio_channels), dtype=torch.float64))
        self.temporal_gated_conv2 = Temporal_Gated_Conv(spatio_channels, output_channels, temporal_kernel_size)
        self.batch_norm = nn.BatchNorm2d(num_nodes)
        self.reset_parameters()

    def reset_parameters(self):
        lim = np.sqrt(1 / self.theta.shape[1])
        self.theta.data.uniform_(-lim, lim)

    def forward(self, A, X):
        """
        :param A: the normalized spatio (n x n) matrix
        :param X: input data with shape (batch_size, num_nodes, num_timesteps, input_channels)
        :return: output data with shape (batch_size, num_nodes, num_timesteps, output_channels)
        """
        temporal_res1 = self.temporal_gated_conv1(X)
        mul_res1 = torch.einsum("mj,ijkl->imkl", A, temporal_res1)
        mul_res2 = torch.einsum("imkl,ln->imkn", mul_res1, self.theta)
        spatio_res = F.relu(mul_res2)
        spatio_res = self.batch_norm(spatio_res)
        temporal_res2 = self.temporal_gated_conv2(spatio_res)
        return temporal_res2
        #normalization_res = self.batch_norm(temporal_res2)
        #return normalization_res


class STGCN(nn.Module):
    """
    Spatio-Temporal Graph Convolutional Network
    """
    def __init__(self, num_timesteps_input, num_timesteps_output, input_channels=1, temporal_kernel_size=3):
        super(STGCN, self).__init__()
        self.st_conv_block1 = ST_Conv_Block(input_channels=input_channels, temporal_channels=64,
                                            spatio_channels=16, output_channels=64,
                                            temporal_kernel_size=temporal_kernel_size)
        self.st_conv_block2 = ST_Conv_Block(input_channels=64, temporal_channels=64,
                                            spatio_channels=16, output_channels=64,
                                            temporal_kernel_size=temporal_kernel_size)
        self.temporal_gated_conv = Temporal_Gated_Conv(input_channels=64, output_channels=64,
                                                       kernel_size=temporal_kernel_size)
        # shape (batch_size, num_nodes, num_timesteps (- (temporal_kernel_size - 1) * (2 + 2 + 1)), 64)
        self.fully_connected = nn.Linear((num_timesteps_input - (temporal_kernel_size - 1) * 5) * 64,
                                         num_timesteps_output)

    def forward(self, A, X):
        """
        :param A: the normalized spatio (n x n) matrix
        :param X: input data with shape (batch_size, num_nodes, num_timesteps_input, input_channels=1)
        :return: output data with shape (batch_size, num_nodes, num_timesteps_out)
        """
        res1 = self.st_conv_block1(A, X)
        res2 = self.st_conv_block2(A, res1)
        res3 = self.temporal_gated_conv(res2)
        res4 = self.fully_connected(res3.reshape((res3.shape[0], res3.shape[1], -1)))
        return res4


In [ ]:
def train_stgcn(X_train, Y_train, batch_size=batch_size):
    permutation = torch.randperm(X_train.shape[0])

    losses = []

    for t in range(0, validation_split_line, batch_size):
        stgcn.train()
        optimizer.zero_grad()

        indices = permutation[t:min(t + batch_size, validation_split_line)]
        X_batch, Y_batch = X_train[indices].to(device=device), Y_train[indices].to(device=device)

        outputs = stgcn(A, X_batch)
        loss = loss_function(outputs, Y_batch)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        stgcn.eval()

    return sum(losses) / len(losses)


def validate_stgcn(X_train, Y_train, batch_size=batch_size):
    permutation = torch.randperm(X_train.shape[0])[validation_split_line:]
    X_validate = X_train[permutation]
    Y_validate = Y_train[permutation]

    losses = []
    rmse_losses = 0
    for t in range(0, X_validate.shape[0], batch_size):
        with torch.no_grad():
            stgcn.eval()
            indices = range(t, min(t + batch_size, X_validate.shape[0]))
            X = X_validate[indices].to(device=device)
            Y = Y_validate[indices].to(device=device)
            outputs = stgcn(A, X)
            loss = loss_function(outputs, Y)
            rmse_loss = np.sum(np.power((outputs.cpu().numpy() - Y.cpu().numpy()) * std, 2))
            losses.append(loss.item())
            rmse_losses += rmse_loss

    return sum(losses) / len(losses), np.sqrt(rmse_losses / np.size(Y_validate.cpu().numpy()))


def predict_stgcn(X_test, mean, std):
    with torch.no_grad():
        stgcn.eval()
        X_test = X_test.to(device=device)
        outputs = stgcn(A, X_test)
    outputs = outputs.cpu().numpy() * std + mean
    return outputs


In [ ]:
def output_to_csv(data):
    assert data.shape == (200, 325, 12)
    data = np.moveaxis(data, 1, -1)
    assert data.shape == (200, 12, 325)
    str_out = "ID,Prediction\n"
    cnt = 0
    for i in range(200): # test example
        for j in range(12): # time steps
            for k in range(325): # node number
                str_out += '%d,%.5f\n' % (cnt, data[i, j, k])
                cnt += 1
    with open('prediction.csv', 'w') as f:
        f.write(str_out)


In [ ]:
stgcn = STGCN(num_timesteps_input, num_timesteps_output).double().to(device=device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(stgcn.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

training_losses = []
validation_losses = []
rmse_losses = []
for epoch in range(epochs):
    loss = train_stgcn(X_train, Y_train)
    validation_loss, rmse_loss = validate_stgcn(X_train, Y_train)
    scheduler.step()

    training_losses.append(loss)
    validation_losses.append(validation_loss)
    rmse_losses.append(rmse_loss)
    print("epoch {}, training loss = {},"
          " validation loss = {}, rmse = {}".format(epoch, loss, validation_loss, rmse_loss))

plt.plot(training_losses, label="Loss")
plt.legend()
plt.xlabel("Epoch Num")
plt.ylabel("Loss")
plt.show()

plt.plot(rmse_losses, label="Validation RMSE Loss")
plt.legend()
plt.xlabel("Epoch Num")
plt.ylabel("RMSE Loss")
plt.show()



In [ ]:
Y_test = predict_stgcn(X_test, mean, std)
output_to_csv(Y_test)
